In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
%%writefile MRJobGraph1.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, sys


class MRJobPrepGraphData(MRJob):
    # run before the mapper processes any input
    def label_dataset(self, _, line):
        
        source_node_id = 1
        # Read dataset and add labels and distances
        splits = line.split("\t")
        node_id = int(splits[0])
        edges = eval(splits[1])
        if (node_id == source_node_id):
            yield str(node_id), str([int(x) for x in edges.keys()]) + "|0|\"\"|Q"
        else:
            yield str(node_id), str([int(x) for x in edges.keys()]) + "|"+str(sys.maxint)+"|\"\"|U"
            
    def steps(self):
        return [MRStep(mapper=self.label_dataset)]

class MRJobGraph1(MRJob):
    def push_frontier(self, _, line):
        self.increment_counter('states', 'Q', 0)
        splits = line.split('\t')
        key = int(splits[0])
        value = splits[1]
        value_splits = value.split('|')
        edges = eval(value_splits[0])
        distance = int(value_splits[1])
        path = eval(value_splits[2])
        status = value_splits[3]
        if status == 'Q':
            for edge in edges:
                self.increment_counter('states', 'Q', 1)
                yield int(edge), '[]|'+str(distance + 1) + '|' + path + str(key) + '|Q'
            status = 'V'
        if status == 'U' or status == 'V':
            yield int(key), str(edges) + '|' + str(distance) + '|' + path + '|' + status
    
    def combine_records(self, key, values):
        final_edges = []
        final_distance = sys.maxint
        final_status = '?'
        final_path = ''
        for value in values:
            value_splits = value.split('|')
            edges = eval(value_splits[0])
            distance = int(value_splits[1])
            path = value_splits[2]
            status = value_splits[3]
            final_edges = final_edges + edges
            final_distance = min(final_distance, distance)
            if final_status == '?' or final_status == 'U':
                final_status = status
            if len(path) > len(final_path):
                final_path = path
        yield key, str(final_edges) + '|' + str(final_distance) + '|' +  final_path + '|' + final_status
        
        
    def passthrough(self, key, value):
        yield key, value
    

    
    def steps(self):
        return [#MRStep(mapper=self.passthrough)
                MRStep(mapper=self.push_frontier,
                       reducer=self.combine_records)
               ]
    
if __name__ == '__main__':
    MRJobGraph1.run()

Overwriting MRJobGraph1.py


In [47]:
from numpy import random,array
from MRJobGraph1 import MRJobGraph1, MRJobPrepGraphData


prep_data = MRJobPrepGraphData(args=['undirected_toy.txt', '--no-strict-protocol'])
with prep_data.make_runner() as runner:
    prepped_data = open("prepped_data0.txt", 'w')
    runner.run()
    for line in runner.stream_output():
        key, value = prep_data.parse_output_line(line)
        print key + "\t" + value
        prepped_data.write(key + '\t' + value + '\n')
    prepped_data.close()
        

print "\n\nSecond Job:\n\n"

iteration = 1
while(iteration < 5):
    mr_job = MRJobGraph1(args=['prepped_data' + str((iteration-1)%2) + '.txt', '--no-strict-protocol'])
    with mr_job.make_runner() as runner: 
        runner.run()
        filename = "prepped_data" + str(iteration%2) + ".txt"
        prepped_data = open(filename, 'w')
        #no output yet
        for line in runner.stream_output():
            key, value = mr_job.parse_output_line(line)
            print str(key) + "\t" + value
            prepped_data.write(str(key) + '\t' + value + '\n')
        prepped_data.close()
        current_counters = runner.counters()
        print current_counters
        print current_counters[0]['states']['Q']
        iteration += 1

1	[2, 5]|0|""|Q
2	[1, 3, 5, 4]|9223372036854775807|""|U
3	[2, 4]|9223372036854775807|""|U
4	[3, 2, 5]|9223372036854775807|""|U
5	[1, 2, 4]|9223372036854775807|""|U


Second Job:


1	[2, 5]|0||V
2	[1, 3, 5, 4]|1|1|Q
3	[2, 4]|9223372036854775807||U
4	[3, 2, 5]|9223372036854775807||U
5	[1, 2, 4]|1|1|Q
[{'states': {'Q': 2}}]
2


SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [2]:
testDict = {'1': 1,'3': 1,'4': 1,'5': 1}

In [4]:
[int(x) for x in testDict.keys()]

[1, 3, 5, 4]

In [8]:
print str(7) + "\t" + str([int(x) for x in testDict.keys()]) + "|0|Q"

7	[1, 3, 5, 4]|0|Q


In [29]:
[] + [] + [2,3,4]

[2, 3, 4]

In [38]:
"test" + "string"

'teststring'

In [45]:
0%2

0